<a href="https://colab.research.google.com/github/Ashwin1999/NLP-project---LDA/blob/main/LDA_on_Wikipedia_Promotional_Articles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# download the excel file and load from drive

# replace this with your drive link to the excel file
location = '/content/drive/My Drive/Colab Notebooks/Dataset Folder 1/Wikipedia Articles.xlsx'

df = pd.read_excel(location)
df = pd.DataFrame(df.iloc[np.random.randint(0, 24027, size=2500), 0])
df.head()

,Articles
10390,Jamstix is a VST instrument produced by Rayzoo...
3827,2012 Koreas 100 Most Influential Person for Ne...
5083,"Gourahari Das is a creative writer, journalist..."
13167,Mayhew formerly known as The Mayhew Animal Hom...
15885,Palampur is a hill station and a municipal cou...


In [ ]:
df.isna().sum()

Articles    0
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(max_df=0.9, min_df=2, stop_words='english')

In [ ]:
dtm = cv.fit_transform(df['Articles'])

In [ ]:
dtm

<2500x31958 sparse matrix of type '<class 'numpy.int64'>'
	with 594942 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
topicCount = 4
LDA = LatentDirichletAllocation(n_components=topicCount)

In [ ]:
LDA.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [ ]:
cv.get_feature_names()[2000]

'aftermarket'

In [ ]:
len(LDA.components_)

4

In [ ]:
LDA.components_.shape

(4, 31958)

In [ ]:
def findTopic(i, topic):
    topic_name = list()
    for t in topic:
        topic_name.append(cv.get_feature_names()[t])
    return f"Topic-{i}:\t{topic_name}\n\n"

i=1
for topic in LDA.components_:
    topic = topic.argsort()[-20:] # get the indices of the top 20 words in each topic
    print(findTopic(i, topic))
    i+=1

Topic-1:	['research', 'women', 'music', 'year', 'state', 'york', 'band', 'president', 'cs1', 'city', 'international', 'released', 'member', 'parser', 'mw', 'output', 'national', 'university', 'album', 'new']


Topic-2:	['institute', 'business', 'department', 'area', 'programs', 'public', 'national', 'development', 'community', 'center', 'year', 'international', 'new', 'college', 'education', 'program', 'research', 'students', 'university', 'school']


Topic-3:	['including', 'time', 'companies', 'management', 'group', 'systems', 'network', 'service', 'use', 'software', 'services', 'data', 'million', 'technology', 'used', 'products', 'business', 'based', 'new', 'company']


Topic-4:	['arts', 'including', 'later', 'book', 'university', 'college', 'festival', 'game', 'american', 'world', 'team', 'years', 'series', 'art', 'year', 'film', 'time', 'music', 'school', 'new']




In [ ]:
topic_results = LDA.transform(dtm)
topic_results.shape

(2500, 4)

In [ ]:
topics_pred = [
    "topic-1",
    "topic-2",
    "topic-3",
    "topic-4",
]

In [ ]:
topic_index = topic_results.argmax(axis=1)
topic_index[:10]

array([2, 0, 3, 1, 1, 2, 3, 2, 2, 1])

In [ ]:
pred = np.array([topics_pred[ind] for ind in topic_index])
pred

array(['topic-3', 'topic-1', 'topic-4', ..., 'topic-1', 'topic-1',
       'topic-4'], dtype='<U7')

In [ ]:
df["Predicted Topics"] = pred

In [ ]:
df.head()

,Articles,Predicted Topics
10390,Jamstix is a VST instrument produced by Rayzoo...,topic-3
3827,2012 Koreas 100 Most Influential Person for Ne...,topic-1
5083,"Gourahari Das is a creative writer, journalist...",topic-4
13167,Mayhew formerly known as The Mayhew Animal Hom...,topic-2
15885,Palampur is a hill station and a municipal cou...,topic-2
